In [4]:
from opyenxes.model.XLog import XLog
from opyenxes.data_in.XUniversalParser import XUniversalParser
from opyenxes.classification.XEventAttributeClassifier import XEventAttributeClassifier

#from keras.models import Sequential
#from keras.layers import Dense
#from keras.layers import Dropout
#from keras.layers import LSTM
#from keras.callbacks import ModelCheckpoint
#from keras.utils import np_utils
import keras

import numpy as np
import itertools

In [6]:
bpic_2011_path = "../logs/bpic2011.xes"

with open(bpic_2011_path) as bpic2011_file:
    bpic2011_log = XUniversalParser().parse(bpic2011_file)[0]

raw_trace = bpic2011_log[0]
raw_event = raw_trace[0]
raw_attributes = raw_event.get_attributes()



Unknown extension: http://www.xes-standard.org/meta_time.xesext
Unknown extension: http://www.xes-standard.org/meta_life.xesext
Unknown extension: http://www.xes-standard.org/meta_org.xesext
Unknown extension: http://www.xes-standard.org/meta_concept.xesext
Unknown extension: http://www.xes-standard.org/meta_3TU.xesext
Unknown extension: http://www.xes-standard.org/meta_general.xesext


In [7]:
event_traces = [[ ev.get_attributes()["concept:name"].get_value() for ev in trace ] for trace in bpic2011_log ]
event_traces = list(itertools.chain.from_iterable(event_traces))

In [21]:
events = sorted(list(set(event_traces)))
event_to_int = dict((c, i) for i, c in enumerate(events))
int_to_event = dict((i,c) for i,c in enumerate(events))

n_events = len(event_traces)
n_vocab = len(events)

print("Total events:", n_events)
print("Total number of event types:", n_vocab)

Total events: 150291
Total number of event types: 624


In [9]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 5
dataX = []
dataY = []

for i in range(0, n_events - seq_length, 1):
    seq_in  = event_traces[i:i + seq_length]
    seq_out = event_traces[i + seq_length]
    dataX.append([event_to_int[ev] for ev in seq_in])
    dataY.append(event_to_int[seq_out])
    
n_patterns = len(dataX)
print("Total Patterns:", n_patterns)

Total Patterns: 150286


In [10]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = keras.utils.np_utils.to_categorical(dataY)

In [11]:
# define the LSTM model
model = keras.models.Sequential()
model.add(keras.layers.LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [16]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
model.fit(X, y, epochs=2, batch_size=128, callbacks=callbacks_list)

Epoch 1/2
 21760/150286 [===>..........................] - ETA: 2:10 - loss: 4.4143

In [12]:
model_filename = "weights-improvement-02-3.3127.hdf5"
model.load_weights(model_filename)

In [13]:
prediction = model.predict(X)

In [22]:
prediction_index = [ np.argmax(p) for p in prediction ]

In [37]:
sum([prediction_index[i] == dataY[i] for i in prediction_index]) / len(prediction_index)

0.2702979652129939